In [18]:
# %load rnn.py
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

class Model:

    def lossFun(inputs, targets, vocab_size, Wxh, Whh, Why, bh, by, hprev):
        """
        inputs,targets are both list of integers.
        hprev is Hx1 array of initial hidden state
        returns the loss, gradients on model parameters, and last hidden state
        """
        xs, hs, ys, ps = {}, {}, {}, {}
        hs[-1] = np.copy(hprev)
        loss = 0
        # forward pass
        for t in range(len(inputs)):
            xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
            xs[t][inputs[t]] = 1
            hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
            ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
            ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
            loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
        # backward pass: compute gradients going backwards
        dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
        dbh, dby = np.zeros_like(bh), np.zeros_like(by)
        dhnext = np.zeros_like(hs[0])
        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t])
            dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
            dWhy += np.dot(dy, hs[t].T)
            dby += dy
            dh = np.dot(Why.T, dy) + dhnext # backprop into h
            dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
            dbh += dhraw
            dWxh += np.dot(dhraw, xs[t].T)
            dWhh += np.dot(dhraw, hs[t-1].T)
            dhnext = np.dot(Whh.T, dhraw)
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
        return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

    def sample(h, seed_ix, n):
        """ 
        sample a sequence of integers from the model 
        h is memory state, seed_ix is seed letter for first time step
        """
        x = np.zeros((vocab_size, 1))
        x[seed_ix] = 1
        ixes = []
        for t in range(n):
            h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
            y = np.dot(Why, h) + by
            p = np.exp(y) / np.sum(np.exp(y))
            ix = np.random.choice(range(vocab_size), p=p.ravel())
            x = np.zeros((vocab_size, 1))
            x[ix] = 1
            ixes.append(ix)
        return ixes


    def sample_top3(h, seed_ix, vocab_size, Wxh, Whh, Why, bh, by):
        """ 
        output the top3 probable next letters 
        h is memory state, seed_ix is seed letter
        """
        x = np.zeros((vocab_size, 1))
        x[seed_ix] = 1
        ixes = []
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ixes = np.random.choice(range(vocab_size),3 , p=p.ravel())
        return ixes
    
    
    def test(data, chars, data_size, vocab_size, char_to_ix, ix_to_char):
        # hyperparameters
        hidden_size = 100 # size of hidden layer of neurons
        seq_length = 25 # number of steps to unroll the RNN for
        learning_rate = 1e-1

        # model parameters
        Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
        Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
        Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
        bh = np.zeros((hidden_size, 1)) # hidden bias
        by = np.zeros((vocab_size, 1)) # output bias

        n, p = 0, 0
        mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
        mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
        smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
        while n<=1000: # 100000
            # prepare inputs (we're sweeping from left to right in steps seq_length long)
            if p+seq_length+1 >= len(data) or n == 0: 
                hprev = np.zeros((hidden_size,1)) # reset RNN memory
                p = 0 # go from start of data
            inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
            targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

            print(inputs)
            print(targets)
            # forward seq_length characters through the net and fetch gradient
            loss, dWxh, dWhh, dWhy, dbh, dby, hprev = Model.lossFun(inputs, targets, vocab_size, Wxh, Whh, Why, bh, by, hprev)
            smooth_loss = smooth_loss * 0.999 + loss * 0.001

            # perform parameter update with Adagrad
            for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                                                            [dWxh, dWhh, dWhy, dbh, dby], 
                                                                            [mWxh, mWhh, mWhy, mbh, mby]):
                mem += dparam * dparam
                param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

            p += seq_length # move data pointer
            n += 1 # iteration counter

        return hprev, Wxh, Whh, Why, bh, by


In [25]:
def load(file):
    data = open('train_input.txt', 'r').read() # should be simple plain text file

    return data

data = load('train_input.txt')
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [26]:
def test(data, chars, data_size, vocab_size, char_to_ix, ix_to_char):
    # hyperparameters
    hidden_size = 100 # size of hidden layer of neurons
    seq_length = 25 # number of steps to unroll the RNN for
    learning_rate = 1e-1

    # model parameters
    Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
    Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
    Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
    bh = np.zeros((hidden_size, 1)) # hidden bias
    by = np.zeros((vocab_size, 1)) # output bias

    n, p = 0, 0
    mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
    smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
    while n<=1000: # 100000
        # prepare inputs (we're sweeping from left to right in steps seq_length long)
        if p+seq_length+1 >= len(data) or n == 0: 
            hprev = np.zeros((hidden_size,1)) # reset RNN memory
            p = 0 # go from start of data
        inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
        targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

        print(inputs)
        print(targets)
        # forward seq_length characters through the net and fetch gradient
        loss, dWxh, dWhh, dWhy, dbh, dby, hprev = Model.lossFun(inputs, targets, vocab_size, Wxh, Whh, Why, bh, by, hprev)
        smooth_loss = smooth_loss * 0.999 + loss * 0.001

        # perform parameter update with Adagrad
        for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                                                        [dWxh, dWhh, dWhy, dbh, dby], 
                                                                        [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

        p += seq_length # move data pointer
        n += 1 # iteration counter

    return hprev, Wxh, Whh, Why, bh, by
    
def save(hprev):
    f_pred = open("pred.txt", "w")

    with open('test_input.txt') as f:
        for line in f:
          line = line.split()
          char = list(line[len(line)-1])
          i = char[len(char)-1] #get last character of input line
          sample_ix = Model.sample_top3(hprev, char_to_ix[i], vocab_size, Wxh, Whh, Why, bh, by)
          txt = ''.join(ix_to_char[ix] for ix in sample_ix)
          f_pred.write(txt + '\n')
    f_pred.close()

In [27]:
hprev, Wxh, Whh, Why, bh, by = test(data, chars, data_size, vocab_size, char_to_ix, ix_to_char)

[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26]
[29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26, 13]
[13, 29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27]
[29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27, 1]
[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29

[26, 4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27]
[4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27, 11]
[11, 29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28]
[29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28, 9]
[9, 16, 4, 29, 27, 22, 11, 26, 4, 20, 13, 26, 22, 16, 5, 4, 27, 14, 2, 29, 20, 22, 16, 5, 4]
[16, 4, 29, 27, 22, 11, 26, 4, 20, 13, 26, 22, 16, 5, 4, 27, 14, 2, 29, 20, 22, 16, 5, 4, 27]
[27, 26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22]
[26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22, 16]
[16, 5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28]
[5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28, 16]
[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27,

[27, 26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22]
[26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22, 16]
[16, 5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28]
[5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28, 16]
[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26]
[29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26, 13]
[13, 29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27]
[29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27, 1]
[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10,

[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10]
[25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10, 2]
[2, 26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27]
[26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27, 26]
[26, 4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27]
[4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27, 11]
[11, 29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 

[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26]
[29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26, 13]
[13, 29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27]
[29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27, 1]
[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29

...

[16, 5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28]
[5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22, 26, 20, 19, 0, 20, 25, 29, 28, 16]
[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26]
[29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26, 13]
[13, 29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27]
[29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27, 1]
[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 2

[7, 29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26]
[29, 18, 17, 29, 27, 22, 11, 26, 16, 4, 29, 15, 27, 26, 27, 13, 26, 11, 10, 29, 0, 20, 28, 26, 13]
[13, 29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27]
[29, 27, 2, 4, 20, 2, 25, 29, 11, 17, 16, 20, 29, 15, 27, 13, 26, 4, 10, 2, 26, 10, 29, 27, 1]
[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29

[1, 5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9]
[5, 15, 29, 13, 27, 12, 9, 16, 28, 9, 27, 10, 29, 4, 29, 11, 26, 22, 29, 20, 27, 28, 5, 9, 29]
[29, 11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27]
[11, 29, 4, 22, 27, 25, 26, 11, 26, 10, 11, 26, 25, 9, 20, 27, 5, 3, 27, 22, 29, 8, 22, 27, 26]
[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10]
[25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10, 2]
[2, 26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27]
[26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27, 26]
[26, 4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 1

[26, 28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22]
[28, 9, 16, 29, 17, 29, 20, 27, 20, 22, 26, 22, 29, 0, 5, 3, 0, 22, 9, 29, 0, 26, 11, 22, 27]
[27, 25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10]
[25, 29, 11, 3, 5, 11, 1, 26, 4, 28, 29, 27, 5, 4, 27, 1, 26, 4, 6, 27, 13, 26, 4, 10, 2]
[2, 26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27]
[26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27, 26]
[26, 4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27]
[4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27, 11]
[11, 29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28]
[29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28, 9]
[9, 16, 4, 29, 27, 22, 11, 26, 4, 20, 13, 26, 22, 16, 5, 4, 27, 14, 2, 

[2, 26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27]
[26, 10, 29, 27, 1, 5, 15, 29, 13, 16, 4, 10, 27, 21, 29, 4, 28, 9, 1, 26, 11, 19, 20, 27, 26]
[26, 4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27]
[4, 15, 27, 25, 29, 11, 3, 5, 11, 1, 20, 27, 11, 2, 15, 16, 1, 29, 4, 22, 26, 11, 6, 27, 11]
[11, 29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28]
[29, 26, 15, 16, 4, 10, 27, 28, 5, 1, 25, 11, 29, 9, 29, 4, 20, 16, 5, 4, 27, 1, 26, 28, 9]
[9, 16, 4, 29, 27, 22, 11, 26, 4, 20, 13, 26, 22, 16, 5, 4, 27, 14, 2, 29, 20, 22, 16, 5, 4]
[16, 4, 29, 27, 22, 11, 26, 4, 20, 13, 26, 22, 16, 5, 4, 27, 14, 2, 29, 20, 22, 16, 5, 4, 27]
[27, 26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22]
[26, 4, 20, 12, 29, 11, 16, 4, 10, 27, 26, 4, 15, 27, 20, 2, 1, 1, 26, 11, 16, 24, 26, 22, 16]
[16, 5, 4, 23, 26, 13, 13, 27, 12, 16, 22, 9, 5, 2, 22, 27, 22,

In [23]:
save(hprev)